In [1]:
from mesmerize_core import *
import numpy as np
from copy import deepcopy
import pandas as pd

2023-01-28 14:33:59.969576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:34:00.136741: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:34:00.136764: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:34:00.803606: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'

**You will need `fastplotlib` installed for the visualizations**

In [2]:
from fastplotlib import ImageWidget
from ipywidgets import VBox, IntSlider, Layout

In [3]:
pd.options.display.max_colwidth = 120

# Paths
These are the only variables you will need to modify in this demo notebook. You will need to set the paths according to your own `caiman_data` dir path

Explanation:

`set_parent_raw_data_path()` - This function from `mesmerize_core` sets the **top level raw data directory**. It is generally the top level directory for your raw experimental data. This allows you to move your experiment directory structure between computers, as long as you keep everything under the parent path the same.

For example,

On Linux based systems if you have your experimental data in the following dir:

`/data/my_name/exp_top_level/....`

You could set `/data/my_name` as the "parent raw data path", and you can then move `exp_top_level/...` between computers.

On windows:

`D:/my_name/exp_top_level/...`

You could set `D:/my_name` as the parent raw data path, and you can then move `exp_top_level/...` between computers.

**Even on windows just use `/`, you do not have to worry about the annoying issue of `\\` and `\` on windows if you use `pathlib` :D**

In [4]:
# for this demo set this dir as the path to your `caiman_data` dir
set_parent_raw_data_path("/home/kushal/caiman_data/")

PosixPath('/home/kushal/caiman_data')

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

**We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems.**

In [5]:
batch_path = get_parent_raw_data_path().joinpath("mesmerize-batch/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [6]:
# create a new batch
df = create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty with the appropriate columns for mesmerize

In [7]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


# Path to the input movie

An input movie must be anywhere within `raw data path` or `batch path`. We will use the Sue 2p example.

In [8]:
# We'll use the Sue movie from caiman
# download it if you don't have it
from caiman.utils.utils import download_demo
download_demo()

'/home/kushal/caiman_data/example_movies/Sue_2x_3000_40_-46.tif'

In [9]:
movie_path = get_parent_raw_data_path().joinpath("example_movies/Sue_2x_3000_40_-46.tif")

# Motion correction parameters

Parameters for all algos have the following structure:

```python
{"main": {... params directly passed to caiman}}
```

In [10]:
# We will start with one version of parameters
mcorr_params1 =\
{
  'main': # this key is necessary for specifying that these are the "main" params for the algorithm
    {
        'max_shifts': [24, 24],
        'strides': [48, 48],
        'overlaps': [24, 24],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [11]:
# add an item to the batch
df.caiman.add_item(
    algo='mcorr',
    input_movie_path=movie_path,
    params=mcorr_params1,
    item_name=movie_path.stem,  # filename of the movie, but can be anything
)

df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,2023-01-28T14:34:43,None,None,None,e3158080-e2be-4706-aa18-93af9ae17f8f


## We can now see that there is one item, a.k.a. row or pandas `Series`, in the batch dataframe, we can add another item with the same input movie but with different parameters.

### **When adding batch items with the same `input_movie_path` (i.e. same input movie but different parameters) it is useful to give them the same `item_name`.**

In [12]:
# We create another set of params, useful for gridsearches for example
mcorr_params2 =\
{
  'main':
    {
        'max_shifts': [24, 24],
        'strides': [24, 24],
        'overlaps': [12, 12],
        'max_deviation_rigid': 3,
        'border_nan': 'copy',
        'pw_rigid': True,
        'gSig_filt': None
    },
}

# add other param variant to the batch
df.caiman.add_item(
  algo='mcorr',
  item_name=movie_path.stem,
  input_movie_path=movie_path,
  params=mcorr_params2
)

df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,2023-01-28T14:34:43,None,None,None,e3158080-e2be-4706-aa18-93af9ae17f8f
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,2023-01-28T14:34:47,None,None,None,46494222-0adb-40a7-9d43-c81e01b47e95


## We can see that there are two batch items for the same input movie.

### We can also use a `for` loop to add multiple different parameter variants more efficiently.

In [13]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(mcorr_params2)

# some variants of max_shifts
for shifts in [1, 6, 12]: 
    # deep copy is the safest way to copy dicts
    new_params = deepcopy(new_params)
    
    # assign the "max_shifts"
    new_params["main"]["max_shifts"] = (shifts, shifts)
    
    df.caiman.add_item(
      algo='mcorr',
      item_name=movie_path.stem,
      input_movie_path=movie_path,
      params=new_params
    )

In [14]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",None,2023-01-28T14:34:43,None,None,None,e3158080-e2be-4706-aa18-93af9ae17f8f
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,2023-01-28T14:34:47,None,None,None,46494222-0adb-40a7-9d43-c81e01b47e95
2,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",None,2023-01-28T14:34:58,None,None,None,81ad6129-2314-4cc9-a1d7-ca7d401d6993
3,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",None,2023-01-28T14:34:58,None,None,None,249713a8-757b-467e-8ae2-4108160378df
4,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",None,2023-01-28T14:34:58,None,None,None,58e8743e-5523-49d7-a005-9cb8119333dc


## Now we can see that there are many parameter variants, but it is not easy to see the differences in parameters between the rows that have the same `item_name`.

### We can use the `caiman.get_params_diffs()` extension to see the unique parameters between rows with the same `item_name`

In [15]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

/tmp/ipykernel_67309/1437176290.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])


0    {'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts': (24, 24)}
1    {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (24, 24)}
2      {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (1, 1)}
3      {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (6, 6)}
4    {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (12, 12)}
Name: params, dtype: object

# Indexing rows and running batch item(s)
#### You can run a single batch item by calling `caiman.run()` on a `Series` (row) of the DataFrame. One way to get the row is integer indexing using `df.iloc[index]`

In [16]:
# get the first batch item
row = df.iloc[0]

### You can see how the various `pandas.Series` extensions are accessible at the level of dataframe rows.

Move the cursor to the end of the following line and press `Tab` on your keyboard. You can select the `caiman.run()` function and press `Shift + Tab` to see the docstring. You can also instead refer to the API docs. https://mesmerize-core.readthedocs.io/en/latest/api/common.html#mesmerize_core.CaimanSeriesExtensions

Note tab completion doesn't work if you use `df.iloc[i].caiman.<method_name>`, you need to apply the indexer to see the docstring.

In [ ]:
row.caiman.

# Run a single batch item

Run the row that we have selected above, on Linux & Mac it will run in subprocess but on Windows it will run in the local kernel. If using the subprocess backend you can use `run(wait=False)` to not block the current kernel when running.

In [17]:
# run the first "batch item"
# this will run in a subprocess by default on Linux & Mac
# on windows it will run locally
process = row.caiman.run()

# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

2023-01-28 14:35:15.707068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:35:15.876402: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:35:15.876432: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:35:16.606391: W tensorflow/compiler/xla/stream_executor/platform/de

starting mc


100%|██████████| 1/1 [00:00<00:00,  7.61it/s]


Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-

# `row.caiman.run()` explained

When you call `row.caiman.run()` it just ends up calling a `run_algo()` function in the appropriate `mesmerize_core.algorithm` modules:

https://github.com/nel-lab/mesmerize-core/tree/master/mesmerize_core/algorithms

By default this function is run in a subprocess on Linux & Mac, but run in the local kernel on Windows (it's a long story...).

For example take a look at `mcorr.py`:

https://github.com/nel-lab/mesmerize-core/blob/master/mesmerize_core/algorithms/mcorr.py

In [ ]:
df.paths.get_batch_path()

In [ ]:
ll 

# Use a loop to run multiple batch items.

`df.iterrows()` iterates through rows and returns the numerical index and row for each iteration

In [18]:
for i, row in df.iterrows():
    if not i > 0: # skip the first item since we've run it already
        continue
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

2023-01-28 14:36:28.563989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:36:28.692860: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:36:28.692885: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:36:29.341877: W tensorflow/compiler/xla/stream_executor/platform/de

starting mc


100%|██████████| 1/1 [00:00<00:00,  5.55it/s]


Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/46494222-0adb-40a7-9d43-c81e01b47e95/46494222-0adb-40a7-9d43-c81e01b47e95-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/46494222-0adb-40a7-9d43-c81e01b47e95/46494222-0adb-40a7-9d43-c81e01b47e95-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/46494222-0adb-40a7-9d43-c81e01b47e95/46494222-0adb-40a7-9d43-c81e01b47e95-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/46494222-0adb-40a7-9d43-c81e01b47e95/46494222-0adb-40a7-9d43-c81e01b47e95-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/46494222-0adb-40a7-9d43-c81e01b47e95/46494222-

2023-01-28 14:37:32.583221: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:37:32.735655: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:37:32.735680: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:37:33.405747: W tensorflow/compiler/xla/stream_executor/platform/de

...

Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-

2023-01-28 14:39:51.430021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:39:51.560173: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:39:51.560197: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:39:52.209177: W tensorflow/compiler/xla/stream_executor/platform/de

starting mc


100%|██████████| 1/1 [00:00<00:00,  8.24it/s]

100%|██████████| 1/1 [00:00<00:00,  7.83it/s]


Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-5523-49d7-a005-9cb8119333dc-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-5523-49d7-a005-9cb8119333dc-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-5523-49d7-a005-9cb8119333dc-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
mc finished successfully!
computing projections
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-5523-49d7-a005-9cb8119333dc-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Computing correlation image
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-

# Reload the DataFrame to see the outputs information for the mcorr batch item
### It is necessary to ALWAYS use `df = df.caiman.reload_from_disk()` after running a single batch item or a loop of batch items. You must not add new batch items until you reload it if you have ran items!

In [19]:
df = df.caiman.reload_from_disk()

Latest## We can see that the `outputs` column has been filled in

In [20]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (48, 48), 'overlaps': (24, 24), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f_mean_projection.n...,2023-01-28T14:34:43,2023-01-28T14:35:48,31.9 sec,None,e3158080-e2be-4706-aa18-93af9ae17f8f
1,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (24, 24), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 46494222-0adb-40a7-9d43-c81e01b47e95/46494222-0adb-40a7-9d43-c81e01b47e95_mean_projection.n...,2023-01-28T14:34:47,2023-01-28T14:37:27,57.89 sec,None,46494222-0adb-40a7-9d43-c81e01b47e95
2,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (1, 1), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 81ad6129-2314-4cc9-a1d7-ca7d401d6993/81ad6129-2314-4cc9-a1d7-ca7d401d6993_mean_projection.n...,2023-01-28T14:34:58,2023-01-28T14:38:36,62.37 sec,None,81ad6129-2314-4cc9-a1d7-ca7d401d6993
3,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df_mean_projection.n...,2023-01-28T14:34:58,2023-01-28T14:39:47,64.96 sec,None,249713a8-757b-467e-8ae2-4108160378df
4,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (12, 12), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan':...",{'mean-projection-path': 58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-5523-49d7-a005-9cb8119333dc_mean_projection.n...,2023-01-28T14:34:58,2023-01-28T14:40:50,58.06 sec,None,58e8743e-5523-49d7-a005-9cb8119333dc


# Check if the algorithm ran successfully for a item

In [21]:
# True if the algo ran succesfully
df.iloc[0]["outputs"]["success"]

True

# Visualization using `fastplotlib`
You will need `fastplotlib` installed for this, see https://github.com/kushalkolar/fastplotlib

# Get the input movie and mcorr so we can visualize them

Note that you DO NOT need to manually work with file paths. For tiff input files it returns it as a memmaped array (if possible) with lazy loading. It will try to use a mesmerize `LazyArray` if the file cannot be memmaped.

In [22]:
# you can change the index to look at the mcorr results of different batch items
index = 0

# get input movie as memmap
input_movie = df.iloc[index].caiman.get_input_movie()

# load mcorr output movie, also as a memmaped array
mcorr_movie = df.iloc[index].mcorr.get_output()

Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap


# Visualize raw & MCorr movie side-by-side

### fastplotlib `ImageWidget` to visualize raw & mcorr movie side by side

`ImageWidget` assumes `"txy"` dimension order by default for 2D movies. You can set other orders using the `dims_order` kwarg

In [23]:
mcorr_iw = ImageWidget(
    data=[input_movie, mcorr_movie], 
    vmin_vmax_sliders=True, 
    cmap="gnuplot2"
)
mcorr_iw.show()

RFBOutputContext()

MESA-INTEL: warning: Performance support disabled, consider sysctl dev.i915.perf_stream_paranoid=0



# Frame averaging with a rolling window using `ImageWidget` "window functions".

## This makes it easier to visually inspect motion

In [25]:
# window function on the "t" (time) dimension, using mean of 17 frames
mcorr_iw.window_funcs = {"t": (np.mean, 17)}

## Close the canvas to free up GPU processing time, not necessary if you have a powerful GPU

In [26]:
mcorr_iw.plot.canvas.close()

## With `ImageWidget` you can view all 5 mcorr results simultaneously!

### This depends on your hard drive's capabilities

In [27]:
# first item is just the raw movie
movies = [df.iloc[0].caiman.get_input_movie()]

# subplot titles
subplot_names = ["raw"]

# we will use the mean images later
means = [df.iloc[0].caiman.get_projection("mean")]

# add all the mcorr outputs to the list
for i, row in df.iterrows():
    # add to the list of movies to plot
    movies.append(row.mcorr.get_output())
    
    # subplot title to show dataframe index
    subplot_names.append(f"ix: {i}")
    
    # mean images which we'll use later
    means.append(row.caiman.get_projection("mean"))

# create the widget
mcorr_iw_multiple = ImageWidget(
    data=movies,  # list of movies
    window_funcs={"t": (np.mean, 17)}, # window_funcs is also a kwarg
    vmin_vmax_sliders=True,
    names=subplot_names,  # subplot names used for titles
    cmap="gnuplot2"
)

mcorr_iw_multiple.show()

Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/e3158080-e2be-4706-aa18-93af9ae17f8f/e3158080-e2be-4706-aa18-93af9ae17f8f-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/46494222-0adb-40a7-9d43-c81e01b47e95/46494222-0adb-40a7-9d43-c81e01b47e95-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/81ad6129-2314-4cc9-a1d7-ca7d401d6993/81ad6129-2314-4cc9-a1d7-ca7d401d6993-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/58e8743e-5523-49d7-a005-9cb8119333dc/58e8743e-5523-49d7-a005-9cb8119333dc-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order

RFBOutputContext()

In [28]:
df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])

/tmp/ipykernel_67309/3787477196.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])


0    {'strides': (48, 48), 'overlaps': (24, 24), 'max_shifts': (24, 24)}
1    {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (24, 24)}
2      {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (1, 1)}
3      {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (6, 6)}
4    {'strides': (24, 24), 'overlaps': (12, 12), 'max_shifts': (12, 12)}
Name: params, dtype: object

### Modify the `window_funcs` at any time

In [29]:
mcorr_iw_multiple.window_funcs["t"].window_size = 5

## There is some motion on the left side of of `ix: 2` at timepoint `1452`, `2037` and a few others. This will be more obvious if we substract a mean image from each frame. You can use `frame_apply` to apply a function before displaying frames in the `ImageWidget`

This can be combined with `window_funcs` or used by itself. If used in combination with `window_funcs`, the window functions are computed first and then fed to `frame_apply`.

For this example the `frame_apply` functions subtract the mean image for each movie.

General form:

```python
{
    data_ix: function() # returns 2D frame
    ...
}
```

In [30]:
subtract_means = {
    0: lambda x: x - means[0],
    1: lambda x: x - means[1],
    2: lambda x: x - means[2],
    3: lambda x: x - means[3],
    4: lambda x: x - means[4],
    5: lambda x: x - means[5]
}

In [31]:
mcorr_iw_multiple.frame_apply = subtract_means

### Different colormaps can make the motion more obvious

In [32]:
for sp in mcorr_iw_multiple.plot:
    sp.graphics[0].cmap = "jet"

In [33]:
# disable frame apply
mcorr_iw_multiple.frame_apply = dict()

# ix `3` seems to work the best so we will cleanup the DataFrame and remove all other items.

### You can remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`

**Note that this also cleans up the output data in the batch directory!**

In [34]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[3].uuid]
rows_keep

['249713a8-757b-467e-8ae2-4108160378df']

### On windows calling `remove_item()` will raise a `PermissionError` if you have the memmap file open.
### Unfortunately the current workaround is to kill the kernel if you want to delete batch items with open memmaps.

There is currently no way to close a `numpy.memmap`, even if you remove all references to it.

In [35]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid)

df

/home/kushal/venvs/workshop/lib/python3.9/site-packages/mesmerize_core/caiman_extensions/common.py:209: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
/home/kushal/venvs/workshop/lib/python3.9/site-packages/mesmerize_core/caiman_extensions/common.py:209: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_children
This feature will change in the future and directly return the  a DataFrame of children (rows, ie. child batch items row) instead of a list of UUIDs

  children = self.get_children(index)
/home/kushal/venvs/workshop/lib/python3.9/site-packages/mesmerize_core/caiman_extensions/common.py

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df_mean_projection.n...,2023-01-28T14:34:58,2023-01-28T14:39:47,64.96 sec,None,249713a8-757b-467e-8ae2-4108160378df


### As you can see above, the numerical index changed for what was previously item 3. Indices are always reset when you use `caiman.remove_item()`. However, UUIDs are always maintained.

# CNMF

## Continue from mcorr above and perform CNMF using the good mcorr output.

First, the params for CNMF. Put the CNMF params within the `main` key, `refit` is if you want to run CNMF for a second iteration.

In [36]:
# some params for CNMF
params_cnmf =\
{
    'main': # indicates that these are the "main" params for the CNMF algo
        {
            'fr': 30, # framerate, very important!
            'p': 1,
            'nb': 2,
            'merge_thr': 0.85,
            'rf': 15,
            'stride': 6, # "stride" for cnmf, "strides" for mcorr
            'K': 4,
            'gSig': [4, 4],
            'ssub': 1,
            'tsub': 1,
            'method_init': 'greedy_roi',
            'min_SNR': 2.0,
            'rval_thr': 0.7,
            'use_cnn': True,
            'min_cnn_thr': 0.8,
            'cnn_lowest': 0.1,
            'decay_time': 0.4,
        },
    'refit': True, # If `True`, run a second iteration of CNMF
}

### Add a single cnmf item to the batch

In [37]:
# add a batch item
df.caiman.add_item(
    algo='cnmf', # algo is cnmf
    input_movie_path=df.iloc[0],  # use mcorr output from a completed batch item
    params=params_cnmf,
    item_name=df.iloc[0]["item_name"], # use the same item name
)

### Just like with motion correction, we can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [38]:
from itertools import product

# variants of several parameters
gSig_variants = [6, 8]
K_variants = [4, 8]
merge_thr_variants = [0.8, 0.95]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmf)

# create a parameter grid
parameter_grid = product(gSig_variants, K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for gSig, K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    new_params_cnmf["main"]["gSig"] = [gSig, gSig]
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmf",
        item_name=df.iloc[0]["item_name"],
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

### See that there are a lot of new cnmf batch items

In [39]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,Sue_2x_3000_40_-46,example_movies/Sue_2x_3000_40_-46.tif,"{'main': {'max_shifts': (6, 6), 'strides': (24, 24), 'overlaps': (12, 12), 'max_deviation_rigid': 3, 'border_nan': '...",{'mean-projection-path': 249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df_mean_projection.n...,2023-01-28T14:34:58,2023-01-28T14:39:47,64.96 sec,None,249713a8-757b-467e-8ae2-4108160378df
1,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",None,2023-01-28T14:43:42,None,None,None,763c9f74-3847-4b01-af9c-550e2f46f2e5
2,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,793c72ed-67b7-42dd-bc6c-70edcaf81cb2
3,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,d752e113-6241-4866-bb96-c293bbd57e95
4,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,801ee5c5-bdbb-4b7b-850c-f57d535eee02
5,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,c0f30b7e-5e6c-4177-81bc-042fc410850a
6,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,e666e0b8-323c-487d-91a8-5f798a1fd15a
7,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,cf2371fd-0fa1-40d9-8d60-c567000f6f4a
8,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,81faf8e9-47e4-4f7a-be02-f808801952ea
9,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,de7cbd40-3e67-4b48-995e-cbf74baed18b


## Since it is difficult to see the different parameter variants above, we can just view the diffs

### The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [40]:
df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])

/tmp/ipykernel_67309/3330105589.py:1: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  df.caiman.get_params_diffs(algo="cnmf", item_name=df.iloc[1]["item_name"])


1    {'gSig': (4, 4), 'merge_thr': 0.85, 'K': 4}
2     {'gSig': (6, 6), 'merge_thr': 0.8, 'K': 4}
3    {'gSig': (6, 6), 'merge_thr': 0.95, 'K': 4}
4     {'gSig': (6, 6), 'merge_thr': 0.8, 'K': 8}
5    {'gSig': (6, 6), 'merge_thr': 0.95, 'K': 8}
6     {'gSig': (8, 8), 'merge_thr': 0.8, 'K': 4}
7    {'gSig': (8, 8), 'merge_thr': 0.95, 'K': 4}
8     {'gSig': (8, 8), 'merge_thr': 0.8, 'K': 8}
9    {'gSig': (8, 8), 'merge_thr': 0.95, 'K': 8}
Name: params, dtype: object

# Run the added `cnmf` batch items

### First, this is how you can filter a pandas DataFrame using multiple columns. This gives you the rows (batch items) using the "cnmf" `"algo"` and those that match a particular `"item_name"`

In [41]:
df[
    (df["algo"] == "cnmf") &  # algo
    (df["item_name"] == df.iloc[0]["item_name"])  # item name
]

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
1,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",None,2023-01-28T14:43:42,None,None,None,763c9f74-3847-4b01-af9c-550e2f46f2e5
2,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,793c72ed-67b7-42dd-bc6c-70edcaf81cb2
3,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,d752e113-6241-4866-bb96-c293bbd57e95
4,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,801ee5c5-bdbb-4b7b-850c-f57d535eee02
5,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,c0f30b7e-5e6c-4177-81bc-042fc410850a
6,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,e666e0b8-323c-487d-91a8-5f798a1fd15a
7,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,cf2371fd-0fa1-40d9-8d60-c567000f6f4a
8,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",None,2023-01-28T14:43:44,None,None,None,81faf8e9-47e4-4f7a-be02-f808801952ea
9,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 't...",None,2023-01-28T14:43:44,None,None,None,de7cbd40-3e67-4b48-995e-cbf74baed18b


## Run only these items

In [42]:
for i, row in df[
    (df["algo"] == "cnmf") &
    (df["item_name"] == df.iloc[0]["item_name"])
].iterrows():
    
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = load_batch(df.paths.get_batch_path())

2023-01-28 14:43:59.053867: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:43:59.185870: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:43:59.185930: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:43:59.841425: W tensorflow/compiler/xla/stream_executor/platform/de

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          Sue_2x_3000_40_-46
input_movie_path    249713a8-757b-467e-8ae2-4108160378df/249713a8-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
added_time                                        2023-01-28T14:43:42
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             763c9f74-3847-4b01-af9c-550e2f46f2e5
Name: 1, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, '

2023-01-28 14:44:52.387531: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-28 14:44:52.387580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: kushal-thinkpad
2023-01-28 14:44:52.387597: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: kushal-thinkpad
2023-01-28 14:44:52.388060: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-01-28 14:44:52.388106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-01-28 14:44:52.388115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-01-28 14:44:52.389948: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optim

performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/kushal/caiman_data/model/cnn_model.json
5/5 [==============================] - 0s 23ms/step
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/763c9f74-3847-4b01-af9c-550e2f46f2e5_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/763c9f74-3847-4b01-af9c-550e2f46f2e5_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/763c9f74-3847-4b01-af9c-550e2f46f2e5_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/763c9f74-3847-4b01-af9c-550e2f46f2e5_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode 

2023-01-28 14:44:59.030329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:44:59.158110: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:44:59.158133: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:44:59.793852: W tensorflow/compiler/xla/stream_executor/platform/de

...

2023-01-28 14:54:48.879500: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-28 14:54:49.027263: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:/home/kushal/venvs/workshop/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-28 14:54:49.027294: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-28 14:54:49.824401: W tensorflow/compiler/xla/stream_executor/platform/de

************************************************************************

Starting CNMF item:
algo                                                             cnmf
item_name                                          Sue_2x_3000_40_-46
input_movie_path    249713a8-757b-467e-8ae2-4108160378df/249713a8-...
params              {'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_th...
outputs                                                          None
added_time                                        2023-01-28T14:43:44
ran_time                                                         None
algo_duration                                                    None
comments                                                         None
uuid                             de7cbd40-3e67-4b48-995e-cbf74baed18b
Name: 9, dtype: object
With params:{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'tsub': 1, 'method_init': 'greedy_roi', 'min_SNR': 2.0, '

Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1_order_F_frames_3000.mmap
making memmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/de7cbd40-3e67-4b48-995e-cbf74baed18b_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap


2023-01-28 14:56:16.932603: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-28 14:56:16.932645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: kushal-thinkpad
2023-01-28 14:56:16.932655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: kushal-thinkpad
2023-01-28 14:56:16.932878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-01-28 14:56:16.932922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-01-28 14:56:16.932935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-01-28 14:56:16.933349: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optim

performing CNMF
fitting images
refitting
performing eval
GPU run not requested, disabling use of GPUs
USING MODEL (keras API): /home/kushal/caiman_data/model/cnn_model.json
12/12 [==============================] - 0s 25ms/step
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/de7cbd40-3e67-4b48-995e-cbf74baed18b_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/de7cbd40-3e67-4b48-995e-cbf74baed18b_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/de7cbd40-3e67-4b48-995e-cbf74baed18b_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decode mmap filename /home/kushal/caiman_data/mesmerize-batch/249713a8-757b-467e-8ae2-4108160378df/de7cbd40-3e67-4b48-995e-cbf74baed18b_cnmf-memmap_d1_170_d2_170_d3_1_order_C_frames_3000.mmap
Decod

### We now have CNMF outputs

In [43]:
df = df.caiman.reload_from_disk()
df[df["algo"] == "cnmf"]

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
1,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.85, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (4, 4), 'ssub': 1, 't...",{'mean-projection-path': 763c9f74-3847-4b01-af9c-550e2f46f2e5/763c9f74-3847-4b01-af9c-550e2f46f2e5_mean_projection.n...,2023-01-28T14:43:42,2023-01-28T14:44:54,53.65 sec,None,763c9f74-3847-4b01-af9c-550e2f46f2e5
2,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 793c72ed-67b7-42dd-bc6c-70edcaf81cb2/793c72ed-67b7-42dd-bc6c-70edcaf81cb2_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:46:13,73.42 sec,None,793c72ed-67b7-42dd-bc6c-70edcaf81cb2
3,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': d752e113-6241-4866-bb96-c293bbd57e95/d752e113-6241-4866-bb96-c293bbd57e95_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:47:32,70.55 sec,None,d752e113-6241-4866-bb96-c293bbd57e95
4,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 'ts...",{'mean-projection-path': 801ee5c5-bdbb-4b7b-850c-f57d535eee02/801ee5c5-bdbb-4b7b-850c-f57d535eee02_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:48:54,76.34 sec,None,801ee5c5-bdbb-4b7b-850c-f57d535eee02
5,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (6, 6), 'ssub': 1, 't...",{'mean-projection-path': c0f30b7e-5e6c-4177-81bc-042fc410850a/c0f30b7e-5e6c-4177-81bc-042fc410850a_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:50:22,81.35 sec,None,c0f30b7e-5e6c-4177-81bc-042fc410850a
6,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': e666e0b8-323c-487d-91a8-5f798a1fd15a/e666e0b8-323c-487d-91a8-5f798a1fd15a_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:51:46,76.99 sec,None,e666e0b8-323c-487d-91a8-5f798a1fd15a
7,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.95, 'rf': 15, 'stride': 6, 'K': 4, 'gSig': (8, 8), 'ssub': 1, 't...",{'mean-projection-path': cf2371fd-0fa1-40d9-8d60-c567000f6f4a/cf2371fd-0fa1-40d9-8d60-c567000f6f4a_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:53:05,72.49 sec,None,cf2371fd-0fa1-40d9-8d60-c567000f6f4a
8,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/249713a8-757b-467e-8ae2-4108160378df-Sue_2x_3000_40_-46_els__d1_170_d2_170_d3_1...,"{'main': {'fr': 30, 'p': 1, 'nb': 2, 'merge_thr': 0.8, 'rf': 15, 'stride': 6, 'K': 8, 'gSig': (8, 8), 'ssub': 1, 'ts...",{'mean-projection-path': 81faf8e9-47e4-4f7a-be02-f808801952ea/81faf8e9-47e4-4f7a-be02-f808801952ea_mean_projection.n...,2023-01-28T14:43:44,2023-01-28T14:54:43,91.38 sec,None,81faf8e9-47e4-4f7a-be02-f808801952ea
9,cnmf,Sue_2x_3000_40_-46,249713a8-757b-467e-8ae2-4108160378df/2

In [45]:
# see which batch items completed succcessfully
df[df["algo"] == "cnmf"]["outputs"].apply(lambda x: x["success"])

1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: outputs, dtype: bool